In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from sklearn.utils import shuffle

import warnings
warnings.filterwarnings("ignore")
pathDrive = 'drive/MyDrive/UNICT/_TESI/dati/'

df = pd.read_csv(pathDrive + 'dati_interpolati_diff_1d-2d_8gg - riordinati.csv', sep=';')

df['edam_ecne_eplu_diff'] = df['edam_ecne_eplu_diff'].str.replace(',','.').astype(float)#
df['edam_ecor_emcn_diff'] = df['edam_ecor_emcn_diff'].str.replace(',','.').astype(float)
df['edam_emsg_eplu_diff'] = df['edam_emsg_eplu_diff'].str.replace(',','.').astype(float)
df['edam_epdn_ecne_diff'] = df['edam_epdn_ecne_diff'].str.replace(',','.').astype(float)
df['edam_epdn_emcn_diff'] = df['edam_epdn_emcn_diff'].str.replace(',','.').astype(float)
df['eint_ecpn_emgl_diff'] = df['eint_ecpn_emgl_diff'].str.replace(',','.').astype(float)#
df['eint_esln_emgl_diff'] = df['eint_esln_emgl_diff'].str.replace(',','.').astype(float)
df['eint_esln_espc_diff'] = df['eint_esln_espc_diff'].str.replace(',','.').astype(float)
df['emeg_ecpn_emgl_diff'] = df['emeg_ecpn_emgl_diff'].str.replace(',','.').astype(float)
df['emfn_ecor_ecri_diff'] = df['emfn_ecor_ecri_diff'].str.replace(',','.').astype(float)
df['emfn_ecor_emcn_diff'] = df['emfn_ecor_emcn_diff'].str.replace(',','.').astype(float)
df['emfn_emcn_epdn_diff'] = df['emfn_emcn_epdn_diff'].str.replace(',','.').astype(float)
df['epdn_ecne_ecpn_diff'] = df['epdn_ecne_ecpn_diff'].str.replace(',','.').astype(float)
df['epdn_eint_ecpn_diff'] = df['epdn_eint_ecpn_diff'].str.replace(',','.').astype(float)##
df['epdn_eint_espc_diff'] = df['epdn_eint_espc_diff'].str.replace(',','.').astype(float)
df['epdn_emfn_espc_diff'] = df['epdn_emfn_espc_diff'].str.replace(',','.').astype(float)
df['eplu_ecne_ecpn_diff'] = df['eplu_ecne_ecpn_diff'].str.replace(',','.').astype(float)
df['eplu_emeg_ecpn_diff'] = df['eplu_emeg_ecpn_diff'].str.replace(',','.').astype(float)
df['eplu_emeg_emal_diff'] = df['eplu_emeg_emal_diff'].str.replace(',','.').astype(float)
df['eplu_emsg_emal_diff'] = df['eplu_emsg_emal_diff'].str.replace(',','.').astype(float)


df_train = df[:126]
df_val = df[126:170]
df_test = df[170:]

y_train = df_train['evento'].astype(int).values
y_val = df_val['evento'].astype(int).values
y_test = df_test['evento'].astype(int).values
#_dftarget = df['evento'].astype(int).values

In [3]:
def toDataFrame(df):
  format = '%d/%m/%Y'
  df['date'] = pd.to_datetime(df['day'], format=format)
  df = df.set_index('date')
  df = df.drop(columns=['day'])
  df = df.drop(columns=['evento'])

  for i in reversed(range(len(df.columns))):
    if df.dtypes[df.columns[i]] == 'object':
      df = df.drop(df.columns[i], axis=1)

  return df

df = toDataFrame(df)
df_train = toDataFrame(df_train)
df_val = toDataFrame(df_val)
df_test = toDataFrame(df_test)

############## normalizzo tutto il dataframe
#normalized_df = (df - df.mean()) / df.std()

############## normalizzo le tre parti del dataset in base alla varianza del training
df_train_mean = df_train.mean()
df_train_std = df_train.std()

normalized_df_train = (df_train - df_train_mean) / df_train_std
normalized_df_val   = (df_val - df_train_mean) / df_train_std
normalized_df_test  = (df_test - df_train_mean) / df_train_std

In [9]:
from sklearn import datasets, decomposition

pca_feature_number = 20
pca = decomposition.PCA(n_components = pca_feature_number)
pca.fit(normalized_df_train) #le PCA si genera a partire dai soli dati del training

def applicaPca(pca_model, normalized_df):
  temp_df = normalized_df.copy()
  new_df = pca_model.transform(temp_df)

  for i in range(pca_model.n_components):
    temp_df['f_' + str(i)] = new_df[:,i]

  return temp_df.filter(like='f_')

df_train_pca = applicaPca(pca, normalized_df_train)
df_val_pca = applicaPca(pca, normalized_df_val)
df_test_pca = applicaPca(pca, normalized_df_test)

#df_test_pca.info()
#pca.explained_variance_ratio_.cumsum()

In [ ]:
'''
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier

#calcola lo score in base all'algoritmo scelto (Random Forest, Logistic RegressionCV, ..) di tutte le prime componenti principali
for i in range(pca_feature_number):
  clf = LogisticRegressionCV(cv=5, random_state=0, class_weight='balanced') #scoring per cambiare metrica
  #clf = RandomForestClassifier(max_depth=5, random_state=0)

  clf.fit(df_train_pca.values[:,:i+1], y_train)

  acc_train = clf.score(df_train_pca.values[:,:i+1],  y_train)*100
  acc_val   = clf.score(df_val_pca.values[:,:i+1],    y_val)*100
  acc_test  = clf.score(df_test_pca.values[:,:i+1],   y_test)*100
  print( str(i+1) + ";" + "{:.1f}".format(acc_train) + ";" + "{:.1f}".format(acc_val) +";" + "{:.1f}".format(acc_test) )
'''

In [ ]:
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
#from sklearn.model_selection import cross_val_score
import numpy as np

#y = _dftarget
#split_at = 150

#max_f1_score = 0
#max_f1_score_who = ""
max_accuracy_train = 0
max_accuracy_val = 0
max_accuracy_tot = 0
max_accuracy_test = 0

max_accuracy_train_who = ""
max_accuracy_val_who = ""
max_accuracy_tot_who = ""
max_accuracy_test_who = ""
contatore = 0

#pca_feature_number = 20 #SOLO SE NON SI USA LA PCA

for i in range(pca_feature_number):
  for j in range(pca_feature_number):
    for a in range(pca_feature_number):
      for b in range(pca_feature_number):
        for c in range(pca_feature_number):
          for d in range(pca_feature_number):
            for e in range(pca_feature_number):
              for f in range(pca_feature_number):
                #for g in range(pca_feature_number):

                  #if i<j and j<a and a<b and b<c and c<d    and i==0 and j==1 and a==2 and b==3 and c==4 and d==5:
                  if i<j and j<a and a<b and b<c and c<d and d<e and e<f:
                    contatore += 1
                    #if contatore > 49778:
                    who = str(contatore) + "; area " + str(i) + " - " + str(j) + " - " + str(a) + " - " + str(b) + " - " + str(c) + " - " + str(d) + " - " + str(e) + " - " + str(f)

                    X_train = df_train_pca.values[:, [i,j,a,b,c,d,e,f]]
                    X_val   = df_val_pca.values[:, [i,j,a,b,c,d,e,f]]
                    X_test  = df_test_pca.values[:, [i,j,a,b,c,d,e,f]]

                    #X_train = X[:split_at]
                    ##X_test = X[split_at:]

                    #y_train = y[:split_at]
                    #y_test = y[split_at:]

                    ################ LOGISTIC REGRESSION with CROSS VALIDATION
                    #clf = LogisticRegressionCV(cv=5, random_state=0, class_weight='balanced') #scoring per cambiare metrica
                    ################ RANDOM FOREST
                    clf = RandomForestClassifier(max_depth=5, random_state=0)

                    clf.fit(X_train, y_train)
                    acc_train = clf.score(X_train,  y_train,  sample_weight=None)*100
                    acc_val   = clf.score(X_val,    y_val,    sample_weight=None)*100
                    acc_test  = clf.score(X_test,   y_test,    sample_weight=None)*100
                    #acc_s = np.mean(cross_val_score(clf, X_train, y_train, cv=5))*100

                    acc_tot = acc_train + acc_val

                    if (max_accuracy_train < acc_train or max_accuracy_val < acc_val or max_accuracy_tot < acc_tot) or (acc_val >= 90) :
                      print( who + ";" + "{:.1f}".format(acc_train) + ";" + "{:.1f}".format(acc_val) +";" + "{:.1f}".format(acc_test) )

                      if max_accuracy_train < acc_train:
                        max_accuracy_train = acc_train
                        max_accuracy_train_who = who

                      if max_accuracy_val < acc_val:
                        max_accuracy_val = acc_val
                        max_accuracy_val_who = who

                      if max_accuracy_tot < acc_tot:
                        max_accuracy_tot = acc_tot


#print( max_f1_score_who )
#print( max_accuracy_score_who )
print( "---------------------> " + max_accuracy_train_who + " " + str(max_accuracy_train) )
print( "---------------------> " + max_accuracy_val_who + " " + str(max_accuracy_val) )
#print( conf_mat )

1; area 0 - 1 - 2 - 3 - 4 - 5 - 6 - 7;92.9;72.7;68.4
2; area 0 - 1 - 2 - 3 - 4 - 5 - 6 - 8;92.9;75.0;68.4
3; area 0 - 1 - 2 - 3 - 4 - 5 - 6 - 9;93.7;65.9;68.4
5; area 0 - 1 - 2 - 3 - 4 - 5 - 6 - 11;95.2;75.0;73.7
6; area 0 - 1 - 2 - 3 - 4 - 5 - 6 - 12;92.9;79.5;68.4
7; area 0 - 1 - 2 - 3 - 4 - 5 - 6 - 13;93.7;81.8;68.4
17; area 0 - 1 - 2 - 3 - 4 - 5 - 7 - 11;97.6;68.2;68.4
134; area 0 - 1 - 2 - 3 - 4 - 6 - 11 - 12;98.4;59.1;73.7
170; area 0 - 1 - 2 - 3 - 4 - 7 - 8 - 9;99.2;52.3;68.4
511; area 0 - 1 - 2 - 3 - 5 - 6 - 12 - 18;93.7;84.1;63.2
